# Dataset 2022 Bereinigung und Normalisierung
## Spezialisiertes Modul für Springer/Immowelt/Immonet Dataset

### Ziel
Bereinigung und Normalisierung des aktuellen Datasets (2022) in ein standardisiertes Format für die gemeinsame Analyse.

### Input
- `data/raw/Dataset_2022.csv`
- `data/processed/berlin_plz_mapping.csv` (PLZ-zu-Bezirk-Mapping)

### Output
- `data/processed/dataset_2022_normalized.csv`

### Besonderheiten
- **PLZ-zu-Bezirk-Mapping erforderlich** (Dataset enthält nur PLZ, keine Bezirksnamen)
- Umfangreiche Spaltenstruktur mit vielen Features
- Deutsche Zahlenformate

### Standardisierte Ausgabespalten
- `price`: Normalisierter Preis (KALTMIETE in €)
- `size`: Normalisierte Größe (WOHNFLAECHE in m²)
- `district`: Berliner Bezirk (via PLZ-Mapping)
- `rooms`: Anzahl Zimmer (ZIMMER)
- `year`: Jahr des Datasets (2022)
- `dataset_id`: Eindeutige Dataset-Kennzeichnung (current)
- `source`: Datenquelle

---
**Teil der modularen Preprocessing-Pipeline**  
**Datum:** 4. Juli 2025  
**Version:** 1.0

## 1. Import Required Libraries

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

# Display configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 20)

print("Bibliotheken erfolgreich importiert!")
print(f"Pandas Version: {pd.__version__}")
print(f"Dataset: 2022 (Springer/Immowelt/Immonet)")

Bibliotheken erfolgreich importiert!
Pandas Version: 2.2.3
Dataset: 2022 (Springer/Immowelt/Immonet)


## 2. PLZ-zu-Bezirk-Mapping laden

In [2]:
# PLZ-zu-Bezirk-Mapping laden
print("=" * 60)
print("PLZ-ZU-BEZIRK-MAPPING LADEN")
print("=" * 60)

try:
    plz_mapping_df = pd.read_csv('data/processed/berlin_plz_mapping.csv')
    print(f"✅ PLZ-Mapping geladen: {len(plz_mapping_df)} Einträge")
    
    # Erstelle Dictionary für schnelles Lookup
    plz_to_district = dict(zip(plz_mapping_df['PLZ'], plz_mapping_df['Bezirk']))
    print(f"✅ PLZ-Dictionary erstellt: {len(plz_to_district)} Zuordnungen")
    
    # Zeige einige Beispiele
    print(f"\nBeispiele:")
    for plz, bezirk in list(plz_to_district.items())[:5]:
        print(f"  {plz} → {bezirk}")
        
    print(f"\nAbgedeckte Bezirke: {len(set(plz_to_district.values()))}")
    print(f"Bezirke: {sorted(set(plz_to_district.values()))}")
    
except FileNotFoundError:
    print("❌ FEHLER: PLZ-Mapping nicht gefunden!")
    print("Bitte stellen Sie sicher, dass 'data/processed/berlin_plz_mapping.csv' existiert.")
    raise

PLZ-ZU-BEZIRK-MAPPING LADEN
✅ PLZ-Mapping geladen: 181 Einträge
✅ PLZ-Dictionary erstellt: 181 Zuordnungen

Beispiele:
  10115 → Mitte
  10117 → Mitte
  10119 → Prenzlauer Berg
  10178 → Mitte
  10179 → Mitte

Abgedeckte Bezirke: 19
Bezirke: ['Charlottenburg', 'Friedrichshain', 'Kreuzberg', 'Lichtenberg', 'Marzahn-Hellersdorf', 'Mitte', 'Neukölln', 'Pankow', 'Prenzlauer Berg', 'Reinickendorf', 'Schöneberg', 'Spandau', 'Steglitz', 'Tempelhof', 'Tiergarten', 'Treptow-Köpenick', 'Wedding', 'Wilmersdorf', 'Zehlendorf']


## 3. Daten laden und erste Analyse

In [3]:
# Lade Dataset 2022
print("=" * 60)
print("DATASET 2022 LADEN UND ANALYSIEREN")
print("=" * 60)

# Lade Rohdaten
df_raw = pd.read_csv('data/raw/Dataset_2022.csv')
print(f"Dataset geladen: {df_raw.shape[0]:,} Zeilen, {df_raw.shape[1]} Spalten")

# Grundlegende Informationen
print(f"\nErste 10 Spalten: {list(df_raw.columns[:10])}")
print(f"Letzte 5 Spalten: {list(df_raw.columns[-5:])}")

# Kernfelder analysieren
core_fields = ['PLZ', 'KALTMIETE', 'WOHNFLAECHE', 'ZIMMER']
print(f"\n=== KERNFELDER ANALYSE ===")
for field in core_fields:
    if field in df_raw.columns:
        non_null = df_raw[field].notna().sum()
        print(f"{field}: {non_null}/{len(df_raw)} ({non_null/len(df_raw)*100:.1f}%) nicht-null")
    else:
        print(f"❌ {field}: Spalte nicht gefunden!")

# PLZ-Analyse
print(f"\n=== PLZ-ANALYSE ===")
unique_plz = df_raw['PLZ'].dropna().unique()
print(f"Einzigartige PLZ: {len(unique_plz)}")
print(f"PLZ-Beispiele: {sorted(unique_plz)[:10]}")

# Erste 5 Zeilen der Kernfelder
print(f"\nErste 5 Zeilen (Kernfelder):")
print(df_raw[core_fields].head())

DATASET 2022 LADEN UND ANALYSIEREN
Dataset geladen: 2,950 Zeilen, 75 Spalten

Erste 10 Spalten: ['ID', 'SORTE', 'PLZ', 'KALTMIETE', 'WARMMIETE', 'NEBENKOSTEN', 'KAUTION', 'HEIZUNGSKOSTEN', 'ZIMMER', 'PARKPLAETZE']
Letzte 5 Spalten: ['Stein', 'Marmor', 'Doppelboden', 'Terracotta', 'Sonstiges']

=== KERNFELDER ANALYSE ===
PLZ: 2950/2950 (100.0%) nicht-null
KALTMIETE: 2772/2950 (94.0%) nicht-null
WOHNFLAECHE: 2950/2950 (100.0%) nicht-null
ZIMMER: 2942/2950 (99.7%) nicht-null

=== PLZ-ANALYSE ===
Einzigartige PLZ: 183
PLZ-Beispiele: [10115, 10117, 10119, 10178, 10179, 10243, 10245, 10247, 10249, 10315]

Erste 5 Zeilen (Kernfelder):
     PLZ  KALTMIETE  WOHNFLAECHE  ZIMMER
0  13125     860.00        73.00     3.0
1  13125     450.28        48.84     2.0
2  13125     739.00        54.79     2.0
3  13125     899.00        74.49     3.0
4  13125     899.00        74.49     3.0


## 4. Spezifische Bereinigung Dataset 2022

In [4]:
# Spezifische Bereinigung für Dataset 2022
print("=" * 60)
print("SPEZIFISCHE BEREINIGUNG DATASET 2022")
print("=" * 60)

# Erstelle Arbeitskopie
df_clean = df_raw.copy()
print(f"Arbeitskopie erstellt: {len(df_clean)} Zeilen")

# === PREIS-BEREINIGUNG (KALTMIETE) ===
print("\n=== PREIS-BEREINIGUNG (KALTMIETE) ===")
print(f"KALTMIETE - Typ: {df_clean['KALTMIETE'].dtype}")
print(f"Nicht-null Werte: {df_clean['KALTMIETE'].notna().sum()}")

# Nur Zeilen mit gültigen Preisen behalten
df_clean = df_clean[df_clean['KALTMIETE'].notna()]
print(f"Nach Entfernung NaN-Preise: {len(df_clean)} Zeilen")

# Unrealistische Preise entfernen (< 100€ oder > 10000€) - Konsistent mit anderen Datasets
initial_count = len(df_clean)
df_clean = df_clean[(df_clean['KALTMIETE'] >= 100) & (df_clean['KALTMIETE'] <= 10000)]
removed_prices = initial_count - len(df_clean)
print(f"Gültige Preise nach Bereinigung: {len(df_clean)}/{initial_count} ({100*len(df_clean)/initial_count:.1f}%)")
print(f"Preisspanne: {df_clean['KALTMIETE'].min():.2f}€ - {df_clean['KALTMIETE'].max():.2f}€")
print(f"Entfernte unrealistische Preise: {removed_prices}")

# === GRÖSSEN-BEREINIGUNG (WOHNFLAECHE) ===
print("\n=== GRÖSSEN-BEREINIGUNG (WOHNFLAECHE) ===")
print(f"WOHNFLAECHE - Typ: {df_clean['WOHNFLAECHE'].dtype}")
print(f"Nicht-null Werte: {df_clean['WOHNFLAECHE'].notna().sum()}")

# Unrealistische Größen entfernen (< 10m² oder > 500m²) - Konsistent mit anderen Datasets
initial_count = len(df_clean)
df_clean = df_clean[(df_clean['WOHNFLAECHE'] >= 10) & (df_clean['WOHNFLAECHE'] <= 500)]
removed_sizes = initial_count - len(df_clean)
print(f"Gültige Größen nach Bereinigung: {len(df_clean)}/{initial_count} ({100*len(df_clean)/initial_count:.1f}%)")
print(f"Größenspanne: {df_clean['WOHNFLAECHE'].min():.1f}m² - {df_clean['WOHNFLAECHE'].max():.1f}m²")
print(f"Entfernte unrealistische Größen: {removed_sizes}")

# === PLZ-ZU-BEZIRK-ZUORDNUNG ===
print("\n=== PLZ-ZU-BEZIRK-ZUORDNUNG ===")
print(f"PLZ - Typ: {df_clean['PLZ'].dtype}")
print(f"Nicht-null PLZ: {df_clean['PLZ'].notna().sum()}")

# WICHTIG: Erweitere das PLZ-Mapping BEVOR die Zuordnung gemacht wird
print("\n=== ERWEITERUNG DES PLZ-MAPPINGS ===")
# Erweitere das PLZ-Dictionary um fehlende PLZ-Codes
additional_plz_mapping = {
    12627: 'Marzahn-Hellersdorf',
    12629: 'Marzahn-Hellersdorf',
    13593: 'Spandau',
    13595: 'Spandau',
    13597: 'Spandau',
    13599: 'Spandau',
    14052: 'Charlottenburg-Wilmersdorf',
    14055: 'Charlottenburg-Wilmersdorf',
    14057: 'Charlottenburg-Wilmersdorf',
    14059: 'Charlottenburg-Wilmersdorf',
    10315: 'Lichtenberg',
    10317: 'Lichtenberg',
    10318: 'Lichtenberg',
    10319: 'Lichtenberg',
    10365: 'Lichtenberg',
    10367: 'Lichtenberg',
    10369: 'Lichtenberg',
    13125: 'Pankow',
    13127: 'Pankow',
    13129: 'Pankow',
    13156: 'Pankow',
    13158: 'Pankow',
    13159: 'Pankow',
    13187: 'Pankow',
    13189: 'Pankow',
    12305: 'Tempelhof-Schöneberg',
    12307: 'Tempelhof-Schöneberg',
    12309: 'Tempelhof-Schöneberg',
    12347: 'Neukölln',
    12349: 'Neukölln',
    12351: 'Neukölln',
    12353: 'Neukölln',
    12355: 'Neukölln',
    12357: 'Neukölln',
    12359: 'Neukölln',
    12681: 'Marzahn-Hellersdorf',
    12683: 'Marzahn-Hellersdorf',
    12685: 'Marzahn-Hellersdorf',
    12687: 'Marzahn-Hellersdorf',
    12689: 'Marzahn-Hellersdorf',
    12679: 'Marzahn-Hellersdorf',
    13051: 'Pankow',
    13053: 'Pankow',
    13055: 'Pankow',
    13057: 'Pankow',
    13059: 'Pankow',
    13086: 'Pankow',
    13088: 'Pankow',
    13089: 'Pankow',
    13403: 'Reinickendorf',
    13405: 'Reinickendorf',
    13407: 'Reinickendorf',
    13409: 'Reinickendorf',
    13435: 'Reinickendorf',
    13437: 'Reinickendorf',
    13439: 'Reinickendorf',
    13465: 'Reinickendorf',
    13467: 'Reinickendorf',
    13469: 'Reinickendorf',
    13503: 'Reinickendorf',
    13505: 'Reinickendorf',
    13507: 'Reinickendorf',
    13509: 'Reinickendorf',
    13581: 'Spandau',
    13583: 'Spandau',
    13585: 'Spandau',
    13587: 'Spandau',
    13589: 'Spandau',
    13591: 'Spandau',
    14195: 'Steglitz-Zehlendorf',
    14197: 'Steglitz-Zehlendorf',
    14199: 'Steglitz-Zehlendorf',
    14163: 'Steglitz-Zehlendorf',
    14165: 'Steglitz-Zehlendorf',
    14167: 'Steglitz-Zehlendorf',
    14169: 'Steglitz-Zehlendorf',
    14129: 'Steglitz-Zehlendorf',
    14109: 'Steglitz-Zehlendorf',
}

# Erweitere das ursprüngliche PLZ-Dictionary
plz_to_district.update(additional_plz_mapping)
print(f"PLZ-Mapping erweitert: {len(plz_to_district)} Zuordnungen")

# Jetzt PLZ zu Bezirk zuordnen
df_clean['district'] = df_clean['PLZ'].map(plz_to_district)
successful_mappings = df_clean['district'].notna().sum()
print(f"Erfolgreiche PLZ-zu-Bezirk-Zuordnungen: {successful_mappings}/{len(df_clean)} ({100*successful_mappings/len(df_clean):.1f}%)")

# Zeige nicht zugeordnete PLZ
unmapped_plz = df_clean[df_clean['district'].isna()]['PLZ'].value_counts().head(10)
if len(unmapped_plz) > 0:
    unique_unmapped = df_clean[df_clean['district'].isna()]['PLZ'].nunique()
    print(f"\nVerbleibende nicht zugeordnete PLZ ({unique_unmapped} einzigartige):")
    for plz, count in unmapped_plz.items():
        print(f"  {plz}: {count} Einträge")

# Nur Zeilen mit gültigen Bezirken behalten
initial_count = len(df_clean)
df_clean = df_clean[df_clean['district'].notna()]
removed_no_district = initial_count - len(df_clean)
print(f"Entfernte Einträge ohne Bezirk: {removed_no_district}")

# === ZIMMER-BEREINIGUNG (ZIMMER) ===
print("\n=== ZIMMER-BEREINIGUNG (ZIMMER) ===")
print(f"ZIMMER - Typ: {df_clean['ZIMMER'].dtype}")
print(f"Nicht-null Werte: {df_clean['ZIMMER'].notna().sum()}")

# Unrealistische Zimmerzahlen entfernen
if len(df_clean) > 0:
    initial_count = len(df_clean)
    df_clean = df_clean[(df_clean['ZIMMER'] >= 1) & (df_clean['ZIMMER'] <= 10)]
    removed_rooms = initial_count - len(df_clean)
    print(f"Gültige Zimmerzahlen nach Bereinigung: {len(df_clean)}/{initial_count} ({100*len(df_clean)/initial_count:.1f}%)")
    if len(df_clean) > 0:
        print(f"Zimmerspanne: {df_clean['ZIMMER'].min():.1f} - {df_clean['ZIMMER'].max():.1f}")
    print(f"Entfernte unrealistische Zimmerzahlen: {removed_rooms}")

print(f"\n✅ Spezifische Bereinigung abgeschlossen")
print(f"Verbleibende Datensätze: {len(df_clean)} (Verlust: {len(df_raw) - len(df_clean)})")

SPEZIFISCHE BEREINIGUNG DATASET 2022
Arbeitskopie erstellt: 2950 Zeilen

=== PREIS-BEREINIGUNG (KALTMIETE) ===
KALTMIETE - Typ: float64
Nicht-null Werte: 2772
Nach Entfernung NaN-Preise: 2772 Zeilen
Gültige Preise nach Bereinigung: 2772/2772 (100.0%)
Preisspanne: 163.31€ - 3000.00€
Entfernte unrealistische Preise: 0

=== GRÖSSEN-BEREINIGUNG (WOHNFLAECHE) ===
WOHNFLAECHE - Typ: float64
Nicht-null Werte: 2772
Gültige Größen nach Bereinigung: 2772/2772 (100.0%)
Größenspanne: 13.0m² - 230.0m²
Entfernte unrealistische Größen: 0

=== PLZ-ZU-BEZIRK-ZUORDNUNG ===
PLZ - Typ: int64
Nicht-null PLZ: 2772

=== ERWEITERUNG DES PLZ-MAPPINGS ===
PLZ-Mapping erweitert: 188 Zuordnungen
Erfolgreiche PLZ-zu-Bezirk-Zuordnungen: 2681/2772 (96.7%)

Verbleibende nicht zugeordnete PLZ (10 einzigartige):
  12247: 16 Einträge
  12207: 16 Einträge
  13627: 12 Einträge
  12203: 11 Einträge
  12209: 9 Einträge
  12279: 7 Einträge
  12277: 7 Einträge
  12249: 6 Einträge
  10551: 4 Einträge
  12205: 3 Einträge
Entfer

## 5. Normalisierung in Standardformat

In [5]:
# Normalisierung in Standardformat
print("=" * 60)
print("NORMALISIERUNG IN STANDARDFORMAT")
print("=" * 60)

# Erstelle normalisiertes Dataset mit Standardspalten
df_normalized = pd.DataFrame()

if len(df_clean) > 0:
    # Standardspalten zuweisen
    df_normalized['price'] = df_clean['KALTMIETE'].astype('float64')
    df_normalized['size'] = df_clean['WOHNFLAECHE'].astype('float64')
    df_normalized['district'] = df_clean['district'].astype('string')
    df_normalized['rooms'] = df_clean['ZIMMER'].astype('float64')
    df_normalized['year'] = 2022
    df_normalized['dataset_id'] = 'current'
    df_normalized['source'] = 'Springer/Immowelt/Immonet'

    # Zusätzliche wichtige Spalten aus Original-Dataset beibehalten
    df_normalized['plz'] = df_clean['PLZ'].astype('string')
    if 'WARMMIETE' in df_clean.columns:
        df_normalized['warmmiete'] = df_clean['WARMMIETE']
    if 'NEBENKOSTEN' in df_clean.columns:
        df_normalized['nebenkosten'] = df_clean['NEBENKOSTEN']
    if 'KAUTION' in df_clean.columns:
        df_normalized['kaution'] = df_clean['KAUTION']
    if 'BAUJAHR' in df_clean.columns:
        df_normalized['baujahr'] = df_clean['BAUJAHR']
    if 'ZUSTAND' in df_clean.columns:
        df_normalized['zustand'] = df_clean['ZUSTAND']
    if 'ENERGIEEFFIZIENSKLASSE' in df_clean.columns:
        df_normalized['energieeffiziensklasse'] = df_clean['ENERGIEEFFIZIENSKLASSE']

    # Ausstattungsmerkmale (boolean)
    ausstattung_cols = ['möbliert', 'Balkon', 'Terrasse', 'Garten', 'Einbauküche', 
                       'Garage', 'Stellplatz', 'Personenaufzug', 'Keller']
    for col in ausstattung_cols:
        if col in df_clean.columns:
            df_normalized[f'ausstattung_{col.lower()}'] = df_clean[col]

print(f"Normalisiertes Dataset erstellt: {len(df_normalized)} Zeilen")
print(f"Standardspalten: {['price', 'size', 'district', 'rooms', 'year', 'dataset_id', 'source']}")
print(f"Zusätzliche Spalten: {len(df_normalized.columns) - 7}")

# Datenqualität prüfen
print(f"\n=== DATENQUALITÄT NORMALISIERTES DATASET ===")
print(f"Zeilen mit Preis: {df_normalized['price'].notna().sum()}")
print(f"Zeilen mit Größe: {df_normalized['size'].notna().sum()}")
print(f"Zeilen mit Bezirk: {df_normalized['district'].notna().sum()}")
print(f"Zeilen mit Zimmeranzahl: {df_normalized['rooms'].notna().sum()}")

# Statistiken
if len(df_normalized) > 0:
    print(f"\n=== STATISTIKEN ===")
    print(f"Preis - Min: {df_normalized['price'].min():.2f}€, Max: {df_normalized['price'].max():.2f}€, Median: {df_normalized['price'].median():.2f}€")
    print(f"Größe - Min: {df_normalized['size'].min():.1f}m², Max: {df_normalized['size'].max():.1f}m², Median: {df_normalized['size'].median():.1f}m²")
    print(f"Zimmer - Min: {df_normalized['rooms'].min():.1f}, Max: {df_normalized['rooms'].max():.1f}, Median: {df_normalized['rooms'].median():.1f}")

    # Bezirksverteilung
    print(f"\n=== BEZIRKSVERTEILUNG ===")
    district_counts = df_normalized['district'].value_counts()
    print(f"Anzahl Bezirke: {len(district_counts)}")
    for district, count in district_counts.head(10).items():
        print(f"  {district}: {count} Einträge")

print(f"\n✅ Normalisierung abgeschlossen!")

NORMALISIERUNG IN STANDARDFORMAT
Normalisiertes Dataset erstellt: 2676 Zeilen
Standardspalten: ['price', 'size', 'district', 'rooms', 'year', 'dataset_id', 'source']
Zusätzliche Spalten: 16

=== DATENQUALITÄT NORMALISIERTES DATASET ===
Zeilen mit Preis: 2676
Zeilen mit Größe: 2676
Zeilen mit Bezirk: 2676
Zeilen mit Zimmeranzahl: 2676

=== STATISTIKEN ===
Preis - Min: 180.00€, Max: 3000.00€, Median: 790.60€
Größe - Min: 13.0m², Max: 230.0m², Median: 65.1m²
Zimmer - Min: 1.0, Max: 5.0, Median: 2.0

=== BEZIRKSVERTEILUNG ===
Anzahl Bezirke: 21
  Spandau: 329 Einträge
  Pankow: 256 Einträge
  Reinickendorf: 247 Einträge
  Treptow-Köpenick: 243 Einträge
  Marzahn-Hellersdorf: 237 Einträge
  Lichtenberg: 151 Einträge
  Neukölln: 147 Einträge
  Friedrichshain: 113 Einträge
  Schöneberg: 107 Einträge
  Mitte: 99 Einträge

✅ Normalisierung abgeschlossen!


## 6. Export des normalisierten Datasets

In [6]:
# Export des normalisierten Datasets
print("=" * 60)
print("EXPORT NORMALISIERTES DATASET")
print("=" * 60)

# Ausgabedatei
output_file = 'data/processed/dataset_2022_normalized.csv'

# Export
df_normalized.to_csv(output_file, index=False)
print(f"✅ Normalisiertes Dataset exportiert: {output_file}")
print(f"Dateigröße: {len(df_normalized)} Zeilen x {len(df_normalized.columns)} Spalten")

# Validierung des Exports
test_load = pd.read_csv(output_file)
print(f"✅ Export-Validierung erfolgreich: {len(test_load)} Zeilen geladen")

# Zusammenfassung
print(f"\n=== ZUSAMMENFASSUNG DATASET 2022 ===")
print(f"Input: data/raw/Dataset_2022.csv ({len(df_raw)} Zeilen)")
print(f"Output: {output_file} ({len(df_normalized)} Zeilen)")
print(f"Datenverlust: {len(df_raw) - len(df_normalized)} Zeilen ({((len(df_raw) - len(df_normalized))/len(df_raw)*100):.1f}%)")
print(f"PLZ-zu-Bezirk-Mapping: {df_normalized['district'].notna().sum()}/{len(df_normalized)} ({df_normalized['district'].notna().sum()/len(df_normalized)*100:.1f}%) erfolgreich")
print(f"Standardisierte Spalten: price, size, district, rooms, year, dataset_id, source")
print(f"Zusätzliche Spalten: {len(df_normalized.columns) - 7}")

print(f"\n🎯 DATASET 2022 BEREINIGUNG ABGESCHLOSSEN!")
print(f"Bereit für Kombination mit anderen normalisierten Datasets.")

EXPORT NORMALISIERTES DATASET
✅ Normalisiertes Dataset exportiert: data/processed/dataset_2022_normalized.csv
Dateigröße: 2676 Zeilen x 23 Spalten
✅ Export-Validierung erfolgreich: 2676 Zeilen geladen

=== ZUSAMMENFASSUNG DATASET 2022 ===
Input: data/raw/Dataset_2022.csv (2950 Zeilen)
Output: data/processed/dataset_2022_normalized.csv (2676 Zeilen)
Datenverlust: 274 Zeilen (9.3%)
PLZ-zu-Bezirk-Mapping: 2676/2676 (100.0%) erfolgreich
Standardisierte Spalten: price, size, district, rooms, year, dataset_id, source
Zusätzliche Spalten: 16

🎯 DATASET 2022 BEREINIGUNG ABGESCHLOSSEN!
Bereit für Kombination mit anderen normalisierten Datasets.


## 7. Lade angereicherte Wohnlagendaten

In [7]:
print("="*60)
print("ANGEREICHERTE WOHNLAGENDATEN LADEN")
print("="*60)

enriched_data_path = 'data/raw/wohnlagen_enriched.csv'
try:
    enriched_df = pd.read_csv(enriched_data_path)
    print(f"✅ Angereicherte Daten geladen: {len(enriched_df):,} Zeilen, {len(enriched_df.columns)} Spalten")
except FileNotFoundError:
    print(f"❌ Datei nicht gefunden: {enriched_data_path}")

ANGEREICHERTE WOHNLAGENDATEN LADEN
✅ Angereicherte Daten geladen: 551,249 Zeilen, 11 Spalten


## 8. Kombiniere Datasets mit Wohnlagendaten

In [12]:
print("="*60)
print("KOMBINIERE MIT WOHNLAGENDATEN")
print("="*60)

# Debug: Check original data sizes
print(f"Original df_normalized: {len(df_normalized):,} Zeilen")
print(f"Original enriched_df: {len(enriched_df):,} Zeilen")

# Create a unique mapping of PLZ to avoid cartesian product
enriched_df_subset = enriched_df[['plz', 'wol', 'ortsteil_neu']].drop_duplicates(subset=['plz'])
print(f"Unique PLZ mappings: {len(enriched_df_subset):,} Zeilen")

# Convert PLZ to string for consistent merge
df_normalized['plz'] = df_normalized['plz'].astype(str)
enriched_df_subset['plz'] = enriched_df_subset['plz'].astype(str)

# Debug: Check PLZ overlap
df_plz_unique = set(df_normalized['plz'].unique())
enriched_plz_unique = set(enriched_df_subset['plz'].unique())
overlap = df_plz_unique.intersection(enriched_plz_unique)
print(f"PLZ overlap: {len(overlap)} von {len(df_plz_unique)} PLZ im Dataset")

# Perform the merge
df_enriched = pd.merge(df_normalized, enriched_df_subset, how='left', on=['plz'])

print(f"✅ Kombiniertes und angereichertes Dataset erstellt: {len(df_enriched):,} Zeilen")

# Check merge success
successful_enrichment = df_enriched['ortsteil_neu'].notna().sum()
print(f"Erfolgreiche Anreicherung: {successful_enrichment:,} von {len(df_enriched):,} Zeilen ({successful_enrichment/len(df_enriched)*100:.1f}%)")

KOMBINIERE MIT WOHNLAGENDATEN
Original df_normalized: 2,676 Zeilen
Original enriched_df: 551,249 Zeilen
Unique PLZ mappings: 193 Zeilen
PLZ overlap: 173 von 173 PLZ im Dataset
✅ Kombiniertes und angereichertes Dataset erstellt: 2,676 Zeilen
Erfolgreiche Anreicherung: 2,676 von 2,676 Zeilen (100.0%)


## 9. Export des finalen angereicherten Datasets

In [13]:
print("="*60)
print("EXPORT FINALES ANGEREICHERTES DATASET")
print("="*60)

# Export
output_file_enriched = 'data/processed/dataset_2022_enriched.csv'
df_enriched.to_csv(output_file_enriched, index=False)

print(f"✅ Finales angereichertes Dataset exportiert: {output_file_enriched}")
print(f"Dateigröße: {len(df_enriched):,} Zeilen x {len(df_enriched.columns)} Spalten")

# Validierung durch Wiedereinlesen
test_df_enriched = pd.read_csv(output_file_enriched)
print(f"✅ Export-Validierung erfolgreich: {len(test_df_enriched):,} Zeilen geladen")

EXPORT FINALES ANGEREICHERTES DATASET
✅ Finales angereichertes Dataset exportiert: data/processed/dataset_2022_enriched.csv
Dateigröße: 2,676 Zeilen x 25 Spalten
✅ Export-Validierung erfolgreich: 2,676 Zeilen geladen
✅ Export-Validierung erfolgreich: 2,676 Zeilen geladen
